# Notebook initialization

##  Global variables

In [1]:
# IPython extension to reload modules before executing user code.
# 'autoreload' reloads modules automatically before entering the execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../")

## Seed initialization (for reproductible results)

In [2]:
import os  
os.environ['PYTHONHASHSEED'] = '0'

# Setting the seed for NumPy generated random numbers
import numpy as np
np.random.seed(0)

# Setting the seed for Python random numbers
import random as rn
rn.seed(0)

# Setting the seed for TensorFlow random numbers
import tensorflow as tf
tf.set_random_seed(0)

## Selecting best GPU to execute on

In [3]:
import os
import gpustat

stats = gpustat.GPUStatCollection.new_query()
ids = map(lambda gpu: int(gpu.entry['index']), stats)
ratios = map(lambda gpu: float(gpu.entry['memory.used'])/float(gpu.entry['memory.total']), stats)
bestGPU = min(zip(ids, ratios), key=lambda x: x[1])[0]

print("setGPU: Setting GPU to: {}".format(bestGPU))
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = str(bestGPU)

setGPU: Setting GPU to: 1


# Importing packages

## User defined packages

In [4]:
from data_utils import import_cifar
from models import VGG16_Vanilla, VGG16

Using TensorFlow backend.


## Other packages

In [5]:
from keras import optimizers
from keras import callbacks
from keras.models import clone_model
from keras.callbacks import EarlyStopping
from collections import defaultdict

# Training model

In [6]:
num_classes = 10
weights = defaultdict(list)
histories = defaultdict(list)

In [7]:
(x_train, y_train), (x_valid, y_valid), (x_test, y_test) = import_cifar(num_classes)

In [8]:
# VGG16 without weight decay nor dropout
vgg16_vanilla = VGG16_Vanilla(input_shape=(32,32,3), num_classes=num_classes)
vgg16_vanilla.save_weights("../weights/initial/VGG16_Vanilla.h5")

weights[vgg16_vanilla] = [[layer.get_weights() for layer in vgg16_vanilla.layers]]

In [9]:
# VGG16 with weight decay and dropout
vgg16 = VGG16(input_shape=(32,32,3), num_classes=num_classes, weight_decay=0.005)
vgg16.save_weights("../weights/initial/VGG16.h5")

weights[vgg16] = [[layer.get_weights() for layer in vgg16.layers]]

In [10]:
def train(model):
    # Training parameters
    batch_size = 1024
    epochs = 100
    lr = 0.1
    
    # Function that is going to be called after each epoch 
    # Decreases the learning rate
    def lr_scheduler(epoch):
        new_lr = lr * (0.9 ** epoch)
        print('new learning rate: {}'.format(new_lr))
        return new_lr
    
    reduce_lr = callbacks.LearningRateScheduler(lr_scheduler)

    # Optimization details
    sgd = optimizers.SGD(lr=lr, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    history = model.fit(x=x_train, 
                        y=y_train, 
                        batch_size=batch_size, 
                        epochs=epochs, 
                        verbose=1, 
                        callbacks=[EarlyStopping(monitor='val_loss', patience=5), reduce_lr],  
                        validation_data=(x_valid, y_valid))
    
    histories[model].append(history)
    weights[model].append([layer.get_weights() for layer in model.layers])

    return model

In [11]:
vgg16_trained = train(clone_model(vgg16))
vgg16_trained.save_weights('../weights/final/VGG16.h5')

Train on 50000 samples, validate on 5000 samples
Epoch 1/100
new learning rate: 0.1
50000/50000 [==============================] - 28s 554us/step - loss: 35.1959 - acc: 0.1914 - val_loss: 28.8257 - val_acc: 0.1202
Epoch 2/100
new learning rate: 0.09000000000000001
50000/50000 [==============================] - 15s 307us/step - loss: 18.5247 - acc: 0.3261 - val_loss: 12.7944 - val_acc: 0.0974
Epoch 3/100
new learning rate: 0.08100000000000002
50000/50000 [==============================] - 15s 304us/step - loss: 8.6139 - acc: 0.3659 - val_loss: 6.7672 - val_acc: 0.1070
Epoch 4/100
new learning rate: 0.0729
50000/50000 [==============================] - 15s 303us/step - loss: 4.6721 - acc: 0.4703 - val_loss: 4.5564 - val_acc: 0.0998
Epoch 5/100
new learning rate: 0.06561
50000/50000 [==============================] - 15s 308us/step - loss: 3.0568 - acc: 0.5393 - val_loss: 3.7510 - val_acc: 0.0994
Epoch 6/100
new learning rate: 0.05904900000000001
50000/50000 [=============================

In [12]:
vgg16_vanilla_trained = train(clone_model(vgg16_vanilla))
vgg16_vanilla_trained.save_weights('../weights/final/VGG16_Vanilla.h5')

Train on 50000 samples, validate on 5000 samples
Epoch 1/100
new learning rate: 0.1
50000/50000 [==============================] - 18s 356us/step - loss: 2.1801 - acc: 0.2652 - val_loss: 7.7788 - val_acc: 0.1458
Epoch 2/100
new learning rate: 0.09000000000000001
50000/50000 [==============================] - 13s 264us/step - loss: 1.6073 - acc: 0.4075 - val_loss: 1.7951 - val_acc: 0.3758
Epoch 3/100
new learning rate: 0.08100000000000002
50000/50000 [==============================] - 13s 266us/step - loss: 1.4169 - acc: 0.4823 - val_loss: 1.5939 - val_acc: 0.4136
Epoch 4/100
new learning rate: 0.0729
50000/50000 [==============================] - 13s 269us/step - loss: 1.2836 - acc: 0.5356 - val_loss: 1.5535 - val_acc: 0.4290
Epoch 5/100
new learning rate: 0.06561
50000/50000 [==============================] - 14s 271us/step - loss: 1.1735 - acc: 0.5792 - val_loss: 1.3795 - val_acc: 0.4958
Epoch 6/100
new learning rate: 0.05904900000000001
50000/50000 [==============================] -